In [1]:
import os 

import tensorflow as tf
import numpy as np
import keras

In [2]:
# Define functional model
inputs = keras.Input(shape=(28, 28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')

dense2 = keras.layers.Dense(10, activation='softmax', name='category_output')
dense3 = keras.layers.Dense(1, activation='sigmoid', name='leftright_output')

In [3]:
x = flatten(inputs)
x = dense1(x)
outputs1 = dense2(x)
outputs2 = dense3(x)

model = keras.Model(inputs=inputs, outputs=[outputs1, outputs2], name='mnist_model')

In [4]:
model.summary()

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 784)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    100,480 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 10)        │      1,290 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leftright_output    │ (None, 1)         │        129 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,899 (398.04 KB)

 Trainable params: 101,899 (398.04 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# loss and optimizer
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy", "accuracy"]

losses = {
    'category_output': loss1,
    'leftright_output': loss2,
}

model.compile(loss=losses, optimizer=optim, metrics=metrics)

In [11]:
# create data with 2 labels
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0 , x_test / 255.0

# 0=left, 1 = right
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)
for idx, y in enumerate(y_train):
    if y>5:
        y_leftright[idx] = 1

print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])

y = {
    "category_output": y_train,
    "leftright_output": y_leftright
}

uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [12]:
# training
model.fit(x_train, y=y, epochs=5,
          batch_size=64, verbose=2)

Epoch 1/5
938/938 - 6s - 6ms/step - category_output_accuracy: 0.9145 - category_output_loss: 0.3060 - leftright_output_accuracy: 0.9304 - leftright_output_loss: 0.1930 - loss: 0.4991
Epoch 2/5
938/938 - 2s - 3ms/step - category_output_accuracy: 0.9579 - category_output_loss: 0.1415 - leftright_output_accuracy: 0.9669 - leftright_output_loss: 0.0976 - loss: 0.2392
Epoch 3/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9695 - category_output_loss: 0.1015 - leftright_output_accuracy: 0.9740 - leftright_output_loss: 0.0769 - loss: 0.1784
Epoch 4/5
938/938 - 2s - 3ms/step - category_output_accuracy: 0.9767 - category_output_loss: 0.0796 - leftright_output_accuracy: 0.9786 - leftright_output_loss: 0.0642 - loss: 0.1437
Epoch 5/5
938/938 - 2s - 2ms/step - category_output_accuracy: 0.9808 - category_output_loss: 0.0637 - leftright_output_accuracy: 0.9812 - leftright_output_loss: 0.0552 - loss: 0.1189


In [13]:
predictions = model.predict(x_test)
len(predictions)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2

In [14]:
prediction_category = predictions[0]
prediction_leftright = predictions[1]

pr_cat = prediction_category[0:20]
pr_lr = prediction_leftright[0:20]

labels_cat = np.argmax(pr_cat, axis=1)
labels_lr = np.array([1 if p >= 0.5 else 0 for p in pr_lr])

In [15]:
print(y_test[0:20])
print(labels_cat[0:20])
print(labels_lr[0:20])

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0]


In [26]:
label = np.argmax(pr_cat, axis=1)

In [27]:
label

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=int64)

In [28]:
pr_cat.shape

(20, 10)